In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from PIL import Image
import os
from sklearn.model_selection import train_test_split 
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
import matplotlib.pyplot as plt
import plotly.express as px
import random
from keras.losses import SparseCategoricalCrossentropy


In [3]:
def fetch_image(data_folder):
    train_data = []
    test_data = []
    valid_data = []
    
    splits = {
        "train": os.path.join(data_folder, "train"),
        "test": os.path.join(data_folder, "test"),
        "valid": os.path.join(data_folder, "valid")
    }
    
    for split_name, split_path in splits.items():
        images_folder = os.path.join(split_path, "images")
        labels_folder = os.path.join(split_path, "labels")
        
        for image_name in os.listdir(images_folder):
            label_name = os.path.splitext(image_name)[0] + ".txt"
            label_path = os.path.join(labels_folder, label_name)
            image_path = os.path.join(images_folder, image_name)
            
            if not os.path.exists(label_path):
                print(f"Warning: Label file not found for {image_name}")
                continue
            
            imgTrafficSignal = Image.open(image_path)
            imgTrafficSignal = imgTrafficSignal.convert("L")
            imgTrafficSignal = imgTrafficSignal.resize((90, 90))
            imgTrafficSignal = np.array(imgTrafficSignal)
            
            with open(label_path, "r") as label_file:
                label_lines = label_file.readlines()
                label_values = []
                for line in label_lines:
                    parts = line.strip().split()
                    class_id = int(parts[0])  # First value is the class ID
                    label_values.append(class_id)
            
            # Use only valid labels (remove -1 or any invalid labels)
            valid_labels = [label for label in label_values if label >= 0]
            
            if not valid_labels:
                print(f"No valid labels for {image_name}")
                continue
            
            # Use the first valid label
            first_label = valid_labels[0]
            
            if split_name == "train":
                train_data.append((imgTrafficSignal, first_label))
            elif split_name == "test":
                test_data.append((imgTrafficSignal, first_label))
            elif split_name == "valid":
                valid_data.append((imgTrafficSignal, first_label))
    
    return train_data, test_data, valid_data

In [15]:
data_folder = "./DataSet"
train_data, test_data, valid_data = fetch_image(data_folder)

No valid labels for FisheyeCamera_1_00802_png.rf.6a502d781364b74f35bfa71832fc1ede.jpg
No valid labels for FisheyeCamera_1_00807_png.rf.065f778b2eaf15dd7b5beb90b30c490c.jpg
No valid labels for FisheyeCamera_1_00926_png.rf.511cfac04ff48a1fe99b560b4a4fb9e2.jpg
No valid labels for FisheyeCamera_1_00922_png.rf.408835a278b9c700eb1a2f358ca58d07.jpg


In [21]:
# Prepare training data features and labels
training_data_features, training_data_labels = zip(*train_data)
training_data_features = np.array(training_data_features)
training_data_labels = np.array(training_data_labels)

# Normalize the images
training_data_features = training_data_features / 255.0

# Reshape for CNN (add channel dimension)
training_data_features = training_data_features.reshape(-1, 90, 90, 1)

In [23]:
cnnModel = Sequential()
cnnModel.add(Conv2D(16,(3,3), padding="same", input_shape=(90, 90, 1), activation='relu'))
print(cnnModel(training_data_features).shape)
cnnModel.add(MaxPool2D((2,2), strides=None, padding="same"))
cnnModel.add(Conv2D(32,(3,3), padding="same", activation='relu'))
cnnModel.add(MaxPool2D((2,2), strides=None, padding="same"))
cnnModel.add(Conv2D(64,(5,5), padding="same", activation='relu'))
cnnModel.add(MaxPool2D((2,2), strides=None, padding="same"))
cnnModel.add(Conv2D(128,(7,7), padding="same", activation='relu'))
cnnModel.add(MaxPool2D((2,2), strides=None, padding="same"))
cnnModel.add(Flatten())
cnnModel.add(Dense(232, activation='relu'))
cnnModel.add(Dense(116, activation='relu'))
cnnModel.add(Dense(58, activation='softmax'))

2024-12-02 21:37:53.199426: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1828396800 exceeds 10% of free system memory.
2024-12-02 21:37:53.884560: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1828396800 exceeds 10% of free system memory.
2024-12-02 21:37:54.367173: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1828396800 exceeds 10% of free system memory.


(3527, 90, 90, 16)


In [25]:
cnnModel.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 90, 90, 16)     │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 45, 45, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 45, 45, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 23, 23, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 23, 23, 64)     │        51,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 12, 12, 128)    │       401,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 232)            │     1,069,288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 116)            │        27,028 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 58)             │         6,786 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,560,702 (5.95 MB)

 Trainable params: 1,560,702 (5.95 MB)

 Non-trainable params: 0 (0.00 B)

In [27]:
cnnModel.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])


In [31]:
# Prepare validation data
validation_data_features, validation_data_labels = zip(*valid_data)
validation_data_features = np.array(validation_data_features)
validation_data_labels = np.array(validation_data_labels)

# Normalize validation images
validation_data_features = validation_data_features / 255.0
validation_data_features = validation_data_features.reshape(-1, 90, 90, 1)

# Train the model
history = cnnModel.fit(
    training_data_features, 
    training_data_labels, 
    validation_data=(validation_data_features, validation_data_labels),
    epochs=12, 
    batch_size=32
)

Epoch 1/12


/home/user/anaconda3/lib/python3.12/site-packages/keras/src/backend/tensorflow/nn.py:678: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


111/111 ━━━━━━━━━━━━━━━━━━━━ 19s 152ms/step - accuracy: 0.0878 - loss: 3.0427 - val_accuracy: 0.1248 - val_loss: 2.5093
Epoch 2/12
111/111 ━━━━━━━━━━━━━━━━━━━━ 17s 152ms/step - accuracy: 0.1638 - loss: 2.5113 - val_accuracy: 0.2497 - val_loss: 2.2929
Epoch 3/12
111/111 ━━━━━━━━━━━━━━━━━━━━ 17s 149ms/step - accuracy: 0.2916 - loss: 2.1907 - val_accuracy: 0.3258 - val_loss: 1.9698
Epoch 4/12
111/111 ━━━━━━━━━━━━━━━━━━━━ 17s 155ms/step - accuracy: 0.3671 - loss: 1.9085 - val_accuracy: 0.3770 - val_loss: 1.8285
Epoch 5/12
111/111 ━━━━━━━━━━━━━━━━━━━━ 17s 154ms/step - accuracy: 0.4181 - loss: 1.7224 - val_accuracy: 0.4182 - val_loss: 1.7414
Epoch 6/12
111/111 ━━━━━━━━━━━━━━━━━━━━ 17s 151ms/step - accuracy: 0.4985 - loss: 1.4482 - val_accuracy: 0.4619 - val_loss: 1.5784
Epoch 7/12
111/111 ━━━━━━━━━━━━━━━━━━━━ 17s 152ms/step - accuracy: 0.5933 - loss: 1.2098 - val_accuracy: 0.5293 - val_loss: 1.5068
Epoch 8/12
111/111 ━━━━━━━━━━━━━━━━━━━━ 16s 146ms/step - accuracy: 0.6923 - loss: 0.9060 - val

NameError: name 'validation_data_features' is not defined